In [1]:
import numpy as np
import pandas as pd
import itertools

## VC Dimension
We will give a brief introduction to the concept of VC Dimension of a hypothesis class. Our main reference text is ```Understanding Machine Learning: From Theory to Algorithms``` by Shai Ben-David and Shai Shalev-Shwartz.

### Preliminaries
- $\mathcal{X}$ is the domain set, where any (training/test) instance $x$ is a member of $\mathcal{X}$.
- $\mathcal{Y}$ is label set, so that every training instance has a known label $y \in \mathcal{Y}$, and we would like to predict the label of the test instances. Assume for simplicity that $\mathcal{Y} = \{ 0,1 \}$.
- $\mathcal{H}$ is the hypothesis class, which is a set of functions $h: \mathcal{X} \mapsto \mathcal{Y}$. 
- $C = \{c_1,\ldots, c_m \} \subset \mathcal{X}$ is a collection of instances. 

#### Definition 1. Restriction of the hypothesis class $\mathcal{H}$ to a set $C$: 
$$\mathcal{H}_C = \{(h(c_1), h(c_1), \ldots, h(c_m) : h \in \mathcal{H} \}.$$
Notice that each function $h_C \in \mathcal{H}_C$ is a point in $\{0,1\}^{|C|}$. 

#### Let us work a little bit on Definition 1 to understand what it really says.
- Step 1: Let $\mathcal{X} = [-5, 5]$ be a closed interval in $\mathbb{R}$.  Let us construct a function $h_{t}(x)$ that returns $1$ if $x \leq t$ and $0$ otherwise. So, the parameter $t$ that defines the function $h$ is a *threshold*, and if the input of $h_t$ is not larger than this threshold, then the function labels this point as $1$. As an example, think as you are classifying lightweight boxers, and if a boxer weighs less than a threshold, you say "yes, this person should be competing in the lightweight boxing category".

In [3]:
def threshold_classifier(t, x):
    return int(x <= t) #first compare x with t, and change the Boolean to an integer
print(threshold_classifier(2,3))
print(threshold_classifier(2,1))

0
1


- Step 2: Let $\mathcal{H}$ be a collection of such $h_t$ for $t = -5, -4.5, -4, \ldots, 4.5, 5$. Formally:
$$\mathcal{H} = \{ h_t : t \in \{-5, -4.5, \ldots, 4.5, 5 \} \}.$$ Assume we have a single instance: $C = \{2.2\}$, that is, we have a single point $x$ to classify. We don't know if the true label of $x$ is $y= 0 $ or $y=1$. But we would like to see, can we *at least* explain both of these cases? This will help us assess the power of our hypothesis class. Print $h_t(X)$ for all $h_t \in \mathcal{H}$.

In [4]:
thresholds = np.arange(-5,5.1, 0.5)
classifications = np.zeros(np.size(thresholds))
for t in range(np.size(thresholds)):
    classifications[t] = threshold_classifier(thresholds[t], 2.2)

In [5]:
df = pd.DataFrame(columns=['threshold','classification'])
df["threshold"] = thresholds
df["classification"] = classifications.astype(int)
print(df.to_string(index=False))

 threshold  classification
      -5.0               0
      -4.5               0
      -4.0               0
      -3.5               0
      -3.0               0
      -2.5               0
      -2.0               0
      -1.5               0
      -1.0               0
      -0.5               0
       0.0               0
       0.5               0
       1.0               0
       1.5               0
       2.0               0
       2.5               1
       3.0               1
       3.5               1
       4.0               1
       4.5               1
       5.0               1


- As we can see, we can "achieve" a classification of 0 or 1, by using *some* functions in $\mathcal{H}$. This is good news, in terms of the following:

`We have enough power to explain any possibe labeling of the data via the hypothesis class we constructed.`

We will soon discuss whether this is a good thing, bad thing, or something that depends on the context.

Question: What is $\mathcal{H}_C$? 

Answer: By definition, as $\mathcal{H}_C$ is a set, it keeps the unique collection of $h_t(2.2)$, which is the following:

In [6]:
"""
df["classification"]: This is indexing or selecting the column named "classification" from the DataFrame df. 
The result is a Pandas Series, which is a one-dimensional array with axis labels.
.unique(): This is a method called on the Pandas Series that returns an array of the unique values found in that Series. 
It essentially filters out any duplicate entries, so each value in the resulting array is distinct. 
The order of the unique values in the resulting array is the order in which they appear in the original Series.
"""

H_C = df["classification"].unique()
print(H_C)

[0 1]


- Step 3: Let us now extend the previous example. Assume $C = \{ 1.1, 2.2 \}$, so we have two instances. Do not confuse this with a single instance with two "columns", as our domain is still restricted to $\mathcal{X} \subset \mathbb{R}$. We can apply the same experiment, but for both of these instances.

In [45]:
thresholds = np.arange(-5,5.1, 0.5)
classifications = np.zeros((np.size(thresholds),),dtype='i,i').tolist()
for t in range(np.size(thresholds)):
    classifications[t] = (threshold_classifier(thresholds[t], 1.1), threshold_classifier(thresholds[t], 2.2))

In [48]:
df = pd.DataFrame(columns=['threshold','classification'])
df["threshold"] = thresholds
df["classification"] = classifications
print(df.to_string(index=False))

 threshold classification
      -5.0         (0, 0)
      -4.5         (0, 0)
      -4.0         (0, 0)
      -3.5         (0, 0)
      -3.0         (0, 0)
      -2.5         (0, 0)
      -2.0         (0, 0)
      -1.5         (0, 0)
      -1.0         (0, 0)
      -0.5         (0, 0)
       0.0         (0, 0)
       0.5         (0, 0)
       1.0         (0, 0)
       1.5         (1, 0)
       2.0         (1, 0)
       2.5         (1, 1)
       3.0         (1, 1)
       3.5         (1, 1)
       4.0         (1, 1)
       4.5         (1, 1)
       5.0         (1, 1)


In [49]:
H_C = df["classification"].unique()
print(H_C)

[(0, 0) (1, 0) (1, 1)]


- We can see that $\mathcal{H}_C$ has three functions inside, where they map $C$ to $(0,0)$, $(1,0)$, and $(1,1)$. You may notice that $(0,1)$ is missing, namely, there is not a function in $\mathcal{H}$ such that we can correctly classify the instances $1.1, \ 2.2$ with labels $0$ and $1$, respectively.

- Step 4: Let us introduce a new classifier, and observe that the 'issue' we have just seen can be resolved by this classifier. Namely, let $h_{l,u}$ classify a point $x$ as $1$ if $l \leq x \leq u$, so that, $l$ is a lower bound threshold and $u$ is an upper bound threshold. Otherwise, let $h_{l,u}$ return $0$.

In [148]:
def interval_classifier(l,u,x): #define the interval classifier
    return int(l <= x and x <= u)
print(interval_classifier(1,3,2))
print(interval_classifier(1,3,-1))

1
0


In [144]:
upper_lower = [element for element in itertools.product(thresholds, thresholds) if element[0] <= element[1]]
#this gives us a collection of (l,u)'s where l,u are both members of "thresholds" and l <= u

In [149]:
classifications = np.zeros((len(upper_lower),),dtype='i,i').tolist() #zero tuples to fill
for t in range(len(upper_lower)): #for all l,u combinations append classifications with the result
    classifications[t] = (interval_classifier(upper_lower[t][0],upper_lower[t][1], 1.1), \
                          interval_classifier(upper_lower[t][0],upper_lower[t][1], 2.2))
unique_tuples = list(set(classifications)) #keep the unique classifications (0,0), (1,0), (0,1), (1,1)
print(unique_tuples)

[(1, 0), (0, 1), (1, 1), (0, 0)]


In [146]:
df = pd.DataFrame(columns=['threshold','classification'])
df["classification"] = unique_tuples #unique classifications
unique_thresholds = np.zeros((len(unique_tuples),),dtype='i,i').tolist()
for search in range(len(unique_tuples)):
    unique_thresholds[search] = [upper_lower[i] for i, tupl in enumerate(classifications) if \
                                 tupl == unique_tuples[search]][0]
df["threshold"] = unique_thresholds #unique classifications
print(df.to_string(index=False))

   threshold classification
 (-5.0, 1.5)         (1, 0)
  (1.5, 2.5)         (0, 1)
 (-5.0, 2.5)         (1, 1)
(-5.0, -5.0)         (0, 0)


- We can see that if our lower bound is $-5$ and upper bound is $1.5$, then we classify the point $1.1$ with a label $1$, since it is between $-5$ and $1.5$. However, we classify $2.2$ as $0$ since it does not fall in this interval. We can see that similarly we can classify these points as $(0,1)$, $(1,1)$, and $(0,0)$. We have $2$ possible labels, and $|C| = 2$ instances, hence there are $2^{|C|}$ many possible labelings, which is equal to $4$ in this case. Hence, the interval classifier can explain every possible labels in this case where $|C| = 2$. Next, let's see if this extends to $|C| = 3$. Assume $C = \{1.1, 2.2, 3.3\}$ for simplicity. (Note that we can also take any possible points in $\mathcal{X} = [-5,5]$, e.g., $C= \{ 1,1,4.3\}$ is also possible.)

In [152]:
classifications = np.zeros((len(upper_lower),),dtype='i,i,i').tolist() #zero tuples to fill
for t in range(len(upper_lower)): #for all l,u combinations append classifications with the result
    classifications[t] = (interval_classifier(upper_lower[t][0],upper_lower[t][1], 1.1),\
                          interval_classifier(upper_lower[t][0],upper_lower[t][1], 2.2), \
                          interval_classifier(upper_lower[t][0],upper_lower[t][1], 3.3))
unique_tuples = list(set(classifications)) #keep the unique classifications (0,0), (1,0), (0,1), (1,1)
print(unique_tuples)

[(1, 1, 0), (0, 1, 0), (0, 0, 0), (1, 0, 0), (0, 0, 1), (1, 1, 1), (0, 1, 1)]


In [154]:
df = pd.DataFrame(columns=['threshold','classification'])
df["classification"] = unique_tuples #unique classifications
unique_thresholds = np.zeros((len(unique_tuples),),dtype='i,i').tolist()
for search in range(len(unique_tuples)):
    unique_thresholds[search] = [upper_lower[i] for i, tupl in enumerate(classifications) if \
                                 tupl == unique_tuples[search]][0]
df["threshold"] = unique_thresholds #unique classifications
print(df.to_string(index=False))
#exercise: make sure to understand this chunk of code

   threshold classification
 (-5.0, 2.5)      (1, 1, 0)
  (1.5, 2.5)      (0, 1, 0)
(-5.0, -5.0)      (0, 0, 0)
 (-5.0, 1.5)      (1, 0, 0)
  (2.5, 3.5)      (0, 0, 1)
 (-5.0, 3.5)      (1, 1, 1)
  (1.5, 3.5)      (0, 1, 1)


- We can see that, we explain $7$ different labelings, so $\mathcal{H}_C$ (recall that this means the hypothesis class $\mathcal{H}$ restricted to the instances $C$) has a cardinality of $7$, or $|\mathcal{H}_C| = 7$. However, we have $2^{|C|} = 2^3 = 8$, and the reason we have $|\mathcal{H}_C| = 7$ is that we can never explain the labeling $(1,0,1)$. This is quite intuitive, since to classify the middle instance as $0$ we should make sure it is not in the specified interval, but to classify the first and last instances we should take them in an interval. Any interval that contains the first and last instances will necessarily contain the middle instance, which makes it impossible to label the three instances as $(1,0,1)$. **It is very important to notice that this result does not only hold for $C = \{1.1, 2.2, 3.3\}$, but any $C = \{x_1, x_2, x_3\}$ where $x_1, x_2, x_3 \in \mathcal{X}$.**

#### Definition 2. $\mathcal{H}$ shatters a finite $C \subset \mathcal{X}$ if $|\mathcal{H}_C |= 2^{|C|}$

- This definition intuitively means that

```Any of the labeling combination of the elements in C can be obtained by a function in our hypothesis class```

So, we discussed that, in the case of $m=1$, the `threshold_classifier(t, x)` *shatters* $C = \{ 2.2 \}$. 
- Exercise: Make sure that for any $x \in \mathcal{X}$, `threshold_classifier(t, x)` shatters $C = \{ x\}$, which means any possible $C \subset \mathcal{X}$ with $|C| = 1$ can be shattered by the threshold classifier. 

Moreover, we demonstrated that `threshold_classifier(t, x)` cannot shatter $C = \{1.1, 2.2\}$. 
- Exercise: Show that this result generalizes for any $C \subset \mathcal{X}$ with $|C| = 2$. In other words, show that, there does not exist any $x_1, x_2 \in \mathcal{X}$ such that $|\mathcal{H}_C |  = 2^{2}$.

Then, we extended our discussion to `interval_classifier(l,u,x)`. For the case of $m=2$ we showed that this classifier shatters $C = \{1.1, 2.2 \}$. 
- Exercise: Show that this result extends to any $C \subset \mathcal{X}$ as long as $|C| = 2$.
- Exercise: Show that, if you show that if $\mathcal{H}$ (or `interval_classifier(l,u,x)` in this context) shatters $C \subset \mathcal{X}$ for $|C| = 2$, then this concludes the same result for all $C \subset \mathcal{X}$ with $|C| = 1$.

Moreover, we showed that `interval_classifier(l,u,x)` does **not** shatter $C = \{1.1, 2.2, 3.3 \}$. 
- Exercise: Show that this result holds for any $C \subset \mathcal{X}$ with $|C| = 3$. 

#### Definition 3. The VC dimension of the hypothesis class $\mathcal{H}$ is the maximum size of $|C|$ such that $C \subseteq \mathcal{X}$ can be shattered by $\mathcal{H}$; that is, the maximum $|C|$ such that $|\mathcal{H}_C| = 2^{|C|}$. If arbitrarily large $C$ can be shattered, we say that $\mathcal{H}$ has an infinite VC dimension.

- A typical strategy to prove the claim that $\mathrm{VCdim}(\mathcal{H}) = m$ is to give an example set $C \subset \mathcal{X}$ with $|C| = m$ that is shattered by $\mathcal{H}$, and then to show that there is no $C \subset \mathcal{X}$ with $|C| \geq m+1$ that can be shattered by $\mathcal{H}$.

- Example 1: Show that the VC dimension of `threshold_classifier(t,x)` is $2$
- Example 2: Show that the VC dimension of `interval_classifier(l,u,x)` is $3$
- Example 3 (new): Discuss that the VC dimension of $\mathcal{H} = \{h_{(a_1,a_2,b_1,b_2)}: a_1 \leq a_2 \text{ and }  b_1 \leq b_2\}$ is equal to $4$ where $h_{(a_1, a_2, b_1, b_2)}$ classifies a point as $1$ if the input $x = [x_1, x_2]$ is in the rectangle: $\{x \ : \ a_1 \leq x_1 \leq a_2 \text{ and }  x_1 \leq x_2\leq  b_2 \}$. Recall that here we have $\mathcal{X} \subset \mathbb{R}^2$.

- Question: Assume $\mathcal{H}$ shatters a set $C$ of size $|C| = 20$. However, we see $10$ points of $C$ as a training set. We find a function from $\mathcal{H}$ by 'learning' the subset of $C$ (that we see as a training set) with respect to the true labels. Since $10 \leq 20$, and since we can shatter $|C| = 20$, then we can definitely find a function in $\mathcal{H}$ that perfectly fits the training set. How confident are you about the performance of this function that was fit to the training set, on the remaining $10$ points? *(Philosophical hint: If someone can explain every phenomenon, his explanations are worthless.)*